In [ ]:
!pip install --upgrade openai requests

from openai import OpenAI
import json
import requests

# set up NVIDIA NIM OpenAI-compatible client
client = OpenAI(
    base_url="https://integrate.api.nvidia.com/v1",
    api_key="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"
)

# input threat scenario
threat_input = """
There is a large looking creature larger than skyscrapers walking toward New York City.
"""

# system prompt for Nemotron model
system_msg = """You are an intelligent emergency response agent trained to handle all types of threat scenarios — including natural disasters, man-made events, public safety issues, biological incidents, and unidentified anomalies — and generate structured emergency response plans.

Return ONLY a valid JSON object with the following fields:

- responders: A list of real-world emergency response groups or units that would logically be called to address this threat. Examples include: Fire, EMS, Police, Animal Control, Military, Public Works, Local Security, Event Staff, Utility Crews, or other official response units. Only include responders that exist in real-world emergency management or municipal/governmental services.

- clarification: A short explanation for why each selected responder was chosen. One line per responder is enough.

- message: A short summary of the threat for first responders (1–2 sentences max).

- priority: One of "High", "Medium", or "Low" depending on urgency, threat level, and public risk.

- actions: A list of specific, realistic, recommended response actions based on the scenario.

STRICT INSTRUCTIONS:
- Do NOT invent fictional or imaginary responder types (e.g., "Alien Division", "Monster Task Force").
- Do NOT generate commentary or explanations outside the JSON.
- If unsure whether a responder is real or relevant, LEAVE IT OUT.
- Output must be valid JSON only. No markdown, no code blocks.
"""

# call the model
response = client.chat.completions.create(
    model="nvidia/llama-3.1-nemotron-ultra-253b-v1",
    messages=[
        {"role": "system", "content": system_msg},
        {"role": "user", "content": threat_input}
    ],
    temperature=0.3
)

# handle model output and strip code block formatting
output = response.choices[0].message.content
print("Raw Output:\n", output)

# clean up possible json code block wrappers
clean_output = output.strip().removeprefix("```json").removesuffix("```").strip()

# attempt to parse JSON
try:
    result = json.loads(clean_output)
    print("\n Parsed JSON Output:\n", json.dumps(result, indent=2))
except json.JSONDecodeError as e:
    print(" JSON parsing failed:", e)
    result = None


# New Section

In [ ]:
# simulated responder endpoints
default_url = "https://httpbin.org/post"
responder_urls = {
    "Fire": default_url,
    "EMS": default_url,
    "Police": default_url,
    "Animal Control": default_url,
    "Local Security": default_url,
    "Event Staff": default_url,
    "Event Manager": default_url,
    "Janitor": default_url,
    "Military": default_url,
    "Public Works": default_url,
    "Utility Crews": default_url
}

# dispatcher function
def dispatch_to_responders(result):
    print("\n Dispatching alerts to responders...\n")

    if not result or "responders" not in result:
        print(" No valid 'responders' field in result.")
        return

    for responder in result["responders"]:
        url = responder_urls.get(responder, default_url)

        payload = {
            "responder": responder,
            "message": result.get("message", "No message provided."),
            "priority": result.get("priority", "Unknown"),
            "actions": result.get("actions", [])
        }

        print(f"\n Sending alert to {responder} → {url}")
        print("Payload:\n", json.dumps(payload, indent=2))

        try:
            response = requests.post(url, json=payload)
            if response.status_code == 200:
                print(f" Successfully sent to {responder}")
            else:
                print(f" Failed to send to {responder} – Status code: {response.status_code}")
        except Exception as e:
            print(f" Error while sending to {responder}: {e}")


In [ ]:
if result:
    dispatch_to_responders(result)